# EDA for AO - Water

In [0]:
# YoY Sales

# %sql
# SELECT YEAR(business_day) AS year_info,
#         ROUND(SUM(amount),0) AS sales
# FROM gold.pos_transactions AS t1
# JOIN gold.material_master AS t2
# ON t1.product_id = t2.material_id
# WHERE business_day BETWEEN '2022-01-01' AND '2023-12-31'
# AND category_name = 'WATER'
# GROUP BY year_info

In [0]:
# Brand-wise Sales Contribution

# %sql
# SELECT brand,
#         ROUND(SUM(amount),0) AS sales,
#         SUM(ROUND(SUM(amount), 0)) OVER () AS total_sales,
#         (sales/total_sales) AS sales_contri
# FROM gold.pos_transactions AS t1
# JOIN gold.material_master AS t2
# ON t1.product_id = t2.material_id
# WHERE business_day BETWEEN '2023-01-01' AND '2023-12-31'
# AND category_name = 'WATER'
# GROUP BY brand

## libs

In [0]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

## Functions

In [0]:
# Function to handle division and replace with 0 in case of errors

def safe_division(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        result = 0
    except FloatingPointError:
        result = 0
    return result

## reading data

In [0]:
cust=pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/ao_cust.csv")
attr=pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/ao_attributes.csv")

gp=pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/ao_gp.csv")

In [0]:
df = pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/ao_products_weekly_data.csv")
df = pd.merge(df, gp[['material_id', 'new_buckets']], on='material_id', how='inner')

##basic eda & data prep

In [0]:
# Calculate total sales for each material and their sales contribution

material_total_sales = df.groupby('material_id')['sale'].sum().reset_index()
df = pd.merge(df, material_total_sales, on='material_id', how='left', suffixes=('', '2'))
df.rename(columns={'sale2': 'material_total_sales'}, inplace=True)

total_sales = df['sale'].sum()
df['sales_contri'] = (df['material_total_sales'] / total_sales)

In [0]:
# Calculate average weekly sales

df['num_weeks'] = df.groupby('material_id')['week_number'].transform('nunique')
df['avg_weekly_sales'] = (df['material_total_sales'] / df['num_weeks']).round()
avg_weekly_sales_df = df[['material_id', 'sales_contri', 'avg_weekly_sales']].drop_duplicates().reset_index(drop = True)

In [0]:
query = """
SELECT store_id, store_name
FROM gold.store_master
"""

store_names = spark.sql(query).toPandas()

In [0]:
material_store_df = pd.read_csv("/dbfs/FileStore/shared_uploads/prem@loyalytics.in/ao_material_store_data.csv")
material_store_df = material_store_df.groupby(['material_id', 'store_id'])['total_sales'].sum().reset_index()

In [0]:
material_store_df['num_materials'] = material_store_df.groupby('store_id')['material_id'].transform('nunique')
material_store_df = material_store_df.groupby('store_id').agg({'total_sales': 'sum', 'num_materials': 'mean'}).reset_index()

material_store_df = material_store_df.merge(store_names, on='store_id', how='left')
material_store_df = material_store_df.sort_values(by = 'num_materials', ascending = False)

material_store_df.to_csv('/dbfs/mnt/cdp-customers/gold-layer/adhoc/assortment_optimization/store_wise_material_listings.csv', index = False)

In [0]:
gp = pd.merge(gp, df[['material_id', 'num_weeks']].drop_duplicates(), on='material_id', how='inner')
gp['avg_weekly_gp'] = gp['GP'] / gp['num_weeks']
gp = pd.merge(gp, avg_weekly_sales_df, on='material_id', how='inner')
gp = pd.merge(gp, attr[['material_id', 'material_name', 'Brand']], on='material_id', how='left')
gp.to_csv('/dbfs/mnt/cdp-customers/gold-layer/adhoc/assortment_optimization/avg_weekly_sales_gp.csv', index = False)

In [0]:
material_store_df.sort_index(inplace = True)
min_total_sales = material_store_df['total_sales'].min()
max_total_sales = material_store_df['total_sales'].max()

In [0]:
ranges = (max_total_sales - min_total_sales)//10
bin_edges = [min_total_sales + ranges*i for i in range(10)]
bin_edges.append(float('inf'))
bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
material_store_df['sales_decile'] = pd.cut(material_store_df['total_sales'],
                                           bins=bin_edges, labels=bin_labels, include_lowest=True)

In [0]:
material_store_df

In [0]:
df = pd.merge(df, attr[['material_id', 'material_name', 'Brand']], on='material_id', how='left')

In [0]:
brand_df = df.groupby(['Brand', 'week_number'])['sale'].sum().reset_index()

brand_total_sales = brand_df.groupby('Brand')['sale'].sum().reset_index()
brand_df = pd.merge(brand_df, brand_total_sales, on='Brand', how='left', suffixes=('', '2'))
brand_df.rename(columns={'sale2': 'brand_total_sales'}, inplace=True)

brand_df['num_weeks'] = brand_df.groupby('Brand')['week_number'].transform('nunique')
brand_df['avg_weekly_sales'] = (brand_df['brand_total_sales'] / brand_df['num_weeks']).round()
brand_avg_weekly_df = brand_df[['Brand', 'avg_weekly_sales']].drop_duplicates().reset_index(drop = True)

In [0]:
brand_gp_df = gp[['Brand', 'GP']].groupby('Brand')['GP'].sum().reset_index()

brand_gp_df = pd.merge(brand_gp_df, brand_df[['Brand', 'num_weeks']].drop_duplicates(), on='Brand', how='inner')
brand_gp_df['avg_weekly_gp'] = brand_gp_df['GP'] / brand_gp_df['num_weeks']
brand_gp_df = pd.merge(brand_gp_df, brand_avg_weekly_df, on='Brand', how='inner')

brand_gp_df.to_csv('/dbfs/mnt/cdp-customers/gold-layer/adhoc/assortment_optimization/brand_avg_weekly_sales_gp.csv', index = False)

In [0]:
cust['vip+freq_perc'] = (cust['vip_cust'] + cust['freq_cust']) / (cust['tot_vip'] + cust['tot_freq'])

In [0]:
data = pd.merge(df, cust[['material_id', 'vip+freq_perc', 'vip_cust_perc', 'freq_cust_perc']], on='material_id', how='left')
data = pd.merge(data, gp[['material_id', 'GP']], on='material_id', how='left')
data = pd.merge(data, attr[['material_id', 'material_group']], on='material_id', how='left')
data = data.drop(columns=['store', 'cwd', 'material_total_sales', 'sales_contri', 'num_weeks', 'avg_weekly_sales', 'Brand'])
data = data[['material_id', 'material_name', 'material_group', 'new_buckets', 'vip+freq_perc', 'vip_cust_perc', 'freq_cust_perc', 'sale', 'vol', 'GP', 'week_number']]

In [0]:
dct = {'Wk' + str(i): [] for i in range(1, 53)}

materials = data['material_id'].unique()
for material in materials:
    weeks = data[data['material_id'] == material]['week_number'].values
    sales = data[data['material_id'] == material]['sale'].values
    vol = data[data['material_id'] == material]['vol'].values
    GP = data[data['material_id'] == material]['GP'].values

    for i, week in enumerate(weeks):
        key = 'Wk' + str(week)
        dct[key].extend([sales[i], vol[i], GP[i]])
    
    for key in dct.keys():
        if int(key[2:]) not in weeks:
            dct[key].extend([0, 0, 0])

converted_df = pd.DataFrame(dct)

materials = [item for item in materials for _ in range(3)]
measures = ['Sales', 'Quantity', 'GP']*312
converted_df['material_id'] = materials
converted_df['measures'] = measures

In [0]:
a = data.drop(columns=['week_number', 'sale', 'vol', 'GP'])
a = a.drop_duplicates()
converted_df = pd.merge(converted_df, a, on='material_id', how='inner')
converted_df.to_csv('/dbfs/mnt/cdp-customers/gold-layer/adhoc/assortment_optimization/data_requirement.csv', index = False)

## final DF

In [0]:
fdf=df.copy()
fdf.head()

## plots

### TLE and SSW

In [0]:
fdf['SSW'] = fdf.apply(lambda row: safe_division(row['sale'], row['store']), axis=1)

pldf = pd.merge(fdf.groupby('material_id')['sale'].sum().reset_index(),
                 fdf[['material_id', 'store', 'new_buckets']].drop_duplicates(),
                 on='material_id',
                 how='inner')
pldf['SSW'] = pldf.apply(lambda row: safe_division(row['sale'], row['store']), axis=1)
pldf = pldf[pldf['new_buckets'] == 'Grow']

fig = px.scatter(pldf, x="SSW", y="store")

fig.add_hline(y=pldf.store.mean())
fig.add_vline(x=pldf.SSW.mean())
fig.show()

In [0]:
temp = fdf.groupby('material_id')['sale'].sum().reset_index()

temp = pd.merge(temp, gp[['material_id', 'GP', 'new_buckets']], on='material_id', how='inner')

temp = pd.merge(temp,
                attr[['material_id', 'material_name','Brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

temp['GP_ABS']=np.abs(temp['GP'])

temp['GP_FLAG']=np.where(temp.GP<0, "Negative", "Positive")

temp = pd.merge(temp, cust[['material_id', 'tot_cust_perc']], on='material_id', how='left')
temp['tot_cust_perc'] = temp['tot_cust_perc'].fillna(0)
temp.rename(columns={'tot_cust_perc': 'cust_pnt'}, inplace=True)

###Sales, GP and Customer penetration

In [0]:
fig = px.scatter(temp, x="sale", y="cust_pnt", size="GP_ABS", 
                 color="GP_FLAG",
                 hover_name="material_name", log_x=False, size_max=40)

fig.add_hline(y=temp.cust_pnt.mean())
fig.add_vline(x=temp.sale.quantile(0.95))
fig.show()

###Delisted view - where they are placed

In [0]:
temp_delist = temp[temp['new_buckets'] == 'Delist']
fig = px.scatter(temp_delist, x="sale", y="cust_pnt", size="GP_ABS", 
                 color="GP_FLAG",
                 hover_name="material_name", log_x=False, size_max=40)

fig.add_hline(y=temp_delist.cust_pnt.mean())
fig.add_vline(x=temp_delist.sale.quantile(0.95))
fig.show()

###Grow view - where they are placed

In [0]:
temp_grow = temp[temp['new_buckets'] == 'Grow']
fig = px.scatter(temp_grow, x="sale", y="cust_pnt", size="GP_ABS", 
                 color="GP_FLAG",
                 hover_name="material_name", log_x=False, size_max=40)

fig.add_hline(y=temp_grow.cust_pnt.mean())
fig.add_vline(x=temp_grow.sale.quantile(0.95))
fig.show()

In [0]:
def get_top_N_list(dataframe, start, end):
    listname = "list" + "_" + str(end)
#     print(listname)
    listname = pd.DataFrame(dataframe.groupby(
        'material_id').sum()['sale']).reset_index().sort_values(
            by=dataframe.columns[2], ascending=False)['material_id'][start:end].tolist()
    return listname

In [0]:
store = fdf[['material_id', 'week_number', 'store']]
sale = fdf[['material_id', 'week_number', 'sale']]
tbl=store[store['material_id'].isin(get_top_N_list(sale,0,16))]

tbl = pd.merge(tbl,
                attr[['material_id', 'material_name','Brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

fig = px.line(tbl, x="week_number", y="store", color='material_name',width=900, height=600)
fig.show()

###Delisted view - their distribution

In [0]:
sale = fdf[fdf['new_buckets'] == 'Delist'][['material_id', 'week_number', 'sale']]

tbl=sale[sale['material_id'].isin(get_top_N_list(sale,0,16))]

tbl = pd.merge(tbl,
                attr[['material_id', 'material_name']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

fig = px.line(tbl, x="week_number", y="sale", color='material_name',width=900, height=600)
fig.show()

In [0]:
sale = fdf[fdf['new_buckets'] == 'Grow'][['material_id', 'week_number', 'sale', 'cwd']]

# listname = "list" + "_" + str(16)
# listname = pd.DataFrame(sale.groupby(
#     'material_id').mean()['cwd']).reset_index().sort_values(
#         by=sale.columns[2], ascending=False)['material_id'][0:16].tolist()

# tbl=sale[sale['material_id'].isin(listname)]
tbl=sale[sale['material_id'].isin(get_top_N_list(sale,0,16))]

tbl = pd.merge(tbl,
                attr[['material_id', 'material_name']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

fig = px.line(tbl, x="week_number", y="cwd", color='material_name',width=900, height=600)
fig.show()

### Brand level

In [0]:
gp_bub2 = pd.merge(fdf,
                attr[['material_id', 'material_name']],
                left_on='material_id',
                right_on='material_id',
                how='inner')


gp_bub=gp_bub2.groupby('Brand').sum()[['sale','vol']].reset_index()

gp2 = pd.merge(gp,
                attr[['material_id', 'material_name','Brand']],
                left_on='material_id',
                right_on='material_id',
                how='inner')

gp3=gp2.groupby('Brand').sum()['GP']

temp=pd.merge(gp_bub,gp3,on='Brand',how='inner')




temp['GP_ABS']=np.abs(temp['GP'])


temp['GP_FLAG']=np.where(temp.GP<0,"Negative","Positive")

In [0]:
fig = px.scatter(temp, x="sale", y="vol", size="GP_ABS", 
                 color="GP_FLAG",
                 hover_name="Brand", log_x=False, size_max=40)

fig.add_hline(y=temp.vol.mean())
fig.add_vline(x=temp.sale.quantile(0.95))


fig.show()
